In [4]:
from Custom_Tetris_Env.tetris import TetrisEnv
from Custom_Tetris_Env.jp import JoypadSpace
from Custom_Tetris_Env.actions import MOVEMENT
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CheckpointCallback
from stable_baselines3.common.monitor import Monitor


# Separate evaluation env
eval_env = TetrisEnv(reward_lines=True, reward_score=True, penalize_height=True)
eval_env = JoypadSpace(eval_env, MOVEMENT)
# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback_stop = EvalCallback(eval_env, callback_on_new_best=callback_on_best, verbose=1)

eval_callback_save = EvalCallback(eval_env, best_model_save_path="./logs/",
                             log_path="./logs/", eval_freq=500,
                             deterministic=True, render=False)
checkpoint = CheckpointCallback(save_freq=100000, save_path="./logs/",
                                name_prefix="dqn_tetris")




env = TetrisEnv(reward_score=True, penalize_height=True)
env = JoypadSpace(env, MOVEMENT)
env = Monitor(env)

model = DQN("MlpPolicy", env, verbose=1, tensorboard_log="./tetris_tensorboard/")

Using cpu device
Wrapping the env in a DummyVecEnv.


In [ ]:
# Train the agent
model.learn(total_timesteps=int(1e5), callback=checkpoint)

: 

In [2]:
env = TetrisEnv()
env = JoypadSpace(env, MOVEMENT)

# load checkpoint
model = DQN.load("logs/dqn_model_1000000_steps.zip", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
# continue training
model.learn(total_timesteps=int(1e6), callback=checkpoint_callback)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.93e+03 |
|    ep_rew_mean      | -19.8    |
|    exploration rate | 0.699    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 640      |
|    time_elapsed     | 49       |
|    total timesteps  | 31720    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.78e+03 |
|    ep_rew_mean      | -19.9    |
|    exploration rate | 0.408    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 623      |
|    time_elapsed     | 99       |
|    total timesteps  | 62272    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.53e-05 |
|    n_updates        | 240566   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean    

In [6]:
done = False
obs = env.reset()


while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()

In [8]:
env.close()